## Evaluating rewards

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
progress = pd.read_csv("/Users/erotokritosskordilis/ray_results/CMAPSS_env/PPO_CMAPSS_env_87800_00000_0_2022-04-19_16-44-38/progress.csv")
progress.head()

In [ ]:
plt.plot(progress.episode_reward_mean)
plt.plot(progress.episode_reward_max, color='green')
plt.plot(progress.episode_reward_min, color='blue')
plt.show()

## Policy evaluation

In [6]:
import pickle

import ray
from ray.tune.registry import register_env
from ray.rllib.agents import ppo
from ray import tune

import tensorflow as tf

from data_prep import DataPrep
from env import CMAPSSEnv
from VAE_dense import VAE

In [7]:
##### Load and configure problem instance #####
file_path = "CMAPSSData/train_FD002.txt"
num_settings = 3
num_sensors = 21
num_units = 20
prev_step_units = 240
step = ""

# Data prep
data = DataPrep(file=file_path,
                num_settings=num_settings, 
                num_sensors=num_sensors, 
                num_units=num_units, 
                prev_step_units=prev_step_units,
                step=step,
                normalization_type="01")

df = data.ReadData()

# List of engine lifetimes
engine_lives = df.groupby(df['Unit']).size()
engine_lives = engine_lives.tolist()
num_engines = len(engine_lives)

print(engine_lives, num_engines)

[183, 273, 230, 128, 253, 194, 197, 234, 202, 184, 266, 135, 149, 260, 340, 163, 309, 143, 205, 316] 20


In [12]:
env_config = {
    "df": df,
    "timestep": 0,
    "obs_size": num_settings+num_sensors+1,
    "engines": num_engines,
    "engine_lives": engine_lives, 
    "decoder_model": tf.keras.models.load_model("/Users/erotokritosskordilis/git-repos/RL_Reliability/saved_models/decoder.h5", compile=False),
}

env_name = "CMAPSS_env"
register_env(env_name, lambda config: CMAPSSEnv(**env_config))

env_config:  {'df':       Unit  NormTime  OpSetting1  OpSetting2  OpSetting3   Sensor1   Sensor2  \
0      241  1.000000    0.595099    0.736342         0.0  0.238089  0.010311   
1      241  0.994536    0.999976    0.997625         1.0  0.000000  0.128061   
2      241  0.989071    0.000038    0.001188         1.0  1.000000  0.985454   
3      241  0.983607    0.833151    0.998694         1.0  0.060269  0.183392   
4      241  0.978142    0.476128    0.831354         1.0  0.626985  0.661941   
...    ...       ...         ...         ...         ...       ...       ...   
4359   260  0.015823    0.476188    0.831354         1.0  0.626985  0.674461   
4360   260  0.012658    0.238102    0.298100         1.0  0.597937  0.647026   
4361   260  0.009494    0.595222    0.736342         0.0  0.238089  0.017952   
4362   260  0.006329    0.595203    0.738717         0.0  0.238089  0.021267   
4363   260  0.003165    0.833260    0.997625         1.0  0.060269  0.194347   

       Sensor3   Se

NotFoundError: 